<a href="https://colab.research.google.com/github/ExeCiety/deepseek-r1-chat/blob/main/Deepseek_R1_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Jan 30 06:11:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -q transformers torch gradio accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Authenticate with Hugging Face
from huggingface_hub import login
login(token="")  # Replace with your actual token

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread
import gradio as gr
import re

# Inisialisasi model dan tokenizer
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"  # Ganti dengan model yang sesuai
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

# Optional manual fallback
if next(model.parameters()).device.type != "cuda":
    model = model.to("cuda")

In [ ]:
def format_message(message, file_content):
    return message

def process_response(text):
    # Cleanup khusus untuk model Mistral
    text = text.replace("</s>", "").strip()
    text = re.sub(r'\n{3,}', '\n\n', text)

    return text


def chat_stream(file_obj, user_message, chat_history):
    try:
        # File handling
        file_content = ""
        if file_obj is not None:
            with open(file_obj.name, 'r', encoding='utf-8') as f:
                file_content = f.read().strip()

        # Format prompt
        prompt = format_message(user_message, file_content)

        # Tokenisasi
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=4096
        ).to(model.device)

        # Streamer config
        streamer = TextIteratorStreamer(
            tokenizer,
            skip_prompt=True,
            skip_special_tokens=False,
            timeout=30
        )

        # Generation parameters
        generation_kwargs = {
            **inputs,
            "streamer": streamer,
            "max_new_tokens": 2048,
            "temperature": 0.3,
            "top_p": 0.9,
            "repetition_penalty": 1.1,
            "do_sample": True,
            "pad_token_id": tokenizer.eos_token_id,
        }

        # Start generation
        thread = Thread(target=model.generate, kwargs=generation_kwargs)
        thread.start()

        # Update chat history
        chat_history.append([user_message, ""])
        accumulated_response = ""

        # Stream handling dengan buffer
        buffer = ""
        for new_token in streamer:
            buffer += new_token
            print(new_token)

            # Proses ketika ada line break atau punctuation
            if re.search(r'[\n.!?]', new_token):
                accumulated_response += buffer
                buffer = ""

                # Process dan update
                processed = process_response(accumulated_response)
                if chat_history[-1][1] != processed:
                    chat_history[-1][1] = processed
                    yield chat_history

        # Final flush buffer
        if buffer:
            accumulated_response += buffer
            processed = process_response(accumulated_response)
            chat_history[-1][1] = processed
            yield chat_history

    except Exception as e:
        error_msg = f"🚨 Error: {str(e)}"
        chat_history[-1][1] = error_msg
        yield chat_history

# CSS Custom
custom_css = ""

# Gradio Interface
with gr.Blocks(theme=gr.themes.Soft(), css=custom_css) as demo:
    gr.Markdown("# 🧠 DeepSeek R1")

    chatbot = gr.Chatbot(
        elem_id="chatbot",
        render_markdown=True,
        latex_delimiters=[
            {"left": "$", "right": "$", "display": False},
            {"left": "$$", "right": "$$", "display": True}
        ]
    )

    with gr.Row():
      file_input = gr.File(
          label="📁 Upload File (TXT only)",
          file_types=['.txt'],
          type="filepath"
      )
      user_input = gr.Textbox(
          placeholder="",
          lines=5,
          max_lines=6,
          label="Prompt"
      )

    submit_btn = gr.Button("🚀 Send", variant="primary")

    submit_btn.click(
        chat_stream,
        inputs=[file_input, user_input, chatbot],
        outputs=chatbot
    )

    user_input.submit(
        chat_stream,
        inputs=[file_input, user_input, chatbot],
        outputs=chatbot
    )

if __name__ == "__main__":
    demo.queue().launch(server_name="0.0.0.0", debug=True)